# Day 2 — Probability & Statistics Basics

In [ ]:
import numpy as np, matplotlib.pyplot as plt
np.random.seed(0)
# Bernoulli samples & mean
p=0.3
x=np.random.binomial(1,p,size=1000)
print('mean ≈', x.mean())
# CLT demo: sample means
means=[np.random.binomial(10,p,size=1000).mean() for _ in range(2000)]
plt.hist(means, bins=40); plt.title('CLT: Binomial(10,p) sample means'); plt.show()

### Bayes rule (symbolic)

In [ ]:
from math import isclose
P_A=0.01; P_B_given_A=0.9; P_B_given_notA=0.05
P_notA=1-P_A
P_B = P_B_given_A*P_A + P_B_given_notA*P_notA
P_A_given_B = P_B_given_A*P_A / P_B
print('P(B)=', round(P_B,4), '  P(A|B)=', round(P_A_given_B,4))